# 3 x Problems

In [1]:
import numpy as np

problem2 = np.load('../data/problem_2.npz')
problem3 = np.load('../data/problem_3.npz')

problem = problem2
x = problem['x']
y = problem['y']

### GP Framework

In [2]:
terminal_set = {'x0': lambda x0: x0, 
                'x1': lambda x1: x1, 
                'x2': lambda x2: x2, 
                '1': 1, 
                '2': 2, 
                '3': 3, 
                '5': 5, 
                '7': 7, 
                'pi': np.pi, 
                'e': np.e}

function_set = {'+': lambda a, b: a+b, 
                '-': lambda a, b: a-b, 
                '*': lambda a, b: a*b,
                '/': lambda a, b: a / b if b != 0 else 1,
                'sin': lambda a: np.sin(a),
                'cos': lambda a: np.cos(a),
                'exp': lambda a: np.exp(a),
                'log': lambda a: np.log(a) if a > 0 else 1,
                'sqrt': lambda a: np.sqrt(a) if a >= 0 else 1,
                'abs': lambda a: np.abs(a)}


def evaluate_tree_array(tree, x0_val, x1_val, x2_val, index=0):
    if(index >= len(tree)):
        return 0
    def subtree_size(tree, index):
        if(index >= len(tree)):
            return 0
        node = tree[index]
        if node in function_set:
            if node in ['sin', 'cos', 'exp', 'log', 'sqrt', 'abs']:
                return 1 + subtree_size(tree, index + 1)
            else:
                left_size = subtree_size(tree, index + 1)
                right_size = subtree_size(tree, index + 1 + left_size)
                return 1 + left_size + right_size
        else:
            return 1

    node = tree[index]
    if node in function_set:
        if node in ['sin', 'cos', 'exp', 'log', 'sqrt', 'abs']:
            left_child_index = index + 1
            return function_set[node](evaluate_tree_array(tree, x0_val, x1_val, x2_val, left_child_index))
        else: 
            left_child_index = index + 1
            left_subtree_size = subtree_size(tree, left_child_index)
            right_child_index = left_child_index + left_subtree_size
            left_value = evaluate_tree_array(tree, x0_val, x1_val, x2_val, left_child_index)
            right_value = evaluate_tree_array(tree, x0_val, x1_val, x2_val, right_child_index)
            return function_set[node](left_value, right_value)
    elif node == 'x0':
        return terminal_set[node](x0_val)
    elif node == 'x1':
        return terminal_set[node](x1_val)
    elif node == 'x2':
        return terminal_set[node](x2_val)
    else: 
        return terminal_set[node]
    

def mean_squared_error(x, y, individual):
    mse = 0
    for i in range(len(y)):
        x0_val = x[0][i]
        x1_val = x[1][i]
        x2_val = x[2][i]
        result = evaluate_tree_array(individual, x0_val, x1_val, x2_val)
        error = np.square(result - y[i])
        mse += error
    mse = mse / len(y)
    return mse

def fitness_function(x, y, individual):
    mse = mean_squared_error(x, y, individual)
    complexity = len(individual)
    regularization = 0.0
    fitness = mse + regularization * complexity
    return fitness

### Population initialization

In [3]:
population_size = 100
max_depth = 5

def random_terminal():
    terminal = np.random.choice(list(terminal_set.keys()))
    return str(terminal)

def random_function():
    function = np.random.choice(list(function_set.keys()))
    return str(function)

def generate_full_tree(max_depth):
    tree = []
    stack = [0] 
    while stack:
        depth = stack.pop()
        if depth < max_depth - 1:
            func = random_function()
            tree.append(func)
            if func in ['sin', 'cos', 'exp', 'log', 'sqrt', 'abs']: 
                stack.append(depth + 1)
            else: 
                stack.append(depth + 1) 
                stack.append(depth + 1) 
        else:
            tree.append(random_terminal())
    return tree

def generate_grow_tree(max_depth):
    tree = []
    stack = [0] 
    while stack:
        depth = stack.pop()
        if depth < max_depth - 1 and (np.random.random() > 0.5 or depth == 0):
            tree.append(random_function())
            stack.append(depth + 1)
            stack.append(depth + 1)
        else:
            tree.append(random_terminal())
    return tree

def initialize_population(population_size, max_depth):
    population = []
    for i in range(population_size):
        if i < population_size // 2:
            individual = generate_full_tree(max_depth)
        else:
            individual = generate_grow_tree(max_depth)
        population.append(individual)
    return population

population = initialize_population(population_size, max_depth)

### Fitness

In [ ]:
def evaluate_population(population):
    population_fitness = []
    for individual in population:
        fitness = fitness_function(x, y, individual)
        population_fitness.append((individual, fitness))
    return population_fitness

population_fitness = evaluate_population(population)

### Evolution

In [ ]:
def selection(population_fitness):
    population_fitness = sorted(population_fitness, key=lambda x: x[1])
    return population_fitness[:population_size // 2]

def crossover(population_fitness, x, y):
    new_population = []
    for i in range(population_size // 4):
        parent1 = population_fitness[np.random.randint(len(population_fitness))][0]
        parent2 = population_fitness[np.random.randint(len(population_fitness))][0]
        child = []
        if len(parent1) < len(parent2):
            for i in range(len(parent1)):
                if np.random.random() > 0.5:
                    child.append(parent1[i])
                else:
                    child.append(parent2[i])
            for j in range(len(parent1), len(parent2)):
                child.append(parent2[j])
        else:
            for i in range(len(parent2)):
                if np.random.random() > 0.5:
                    child.append(parent1[i])
                else:
                    child.append(parent2[i])
            for j in range(len(parent2), len(parent1)):
                child.append(parent1[j])
        new_population.append((child, fitness_function(x, y, child)))
    return new_population

def mutation(population_fitness, x, y):
    new_population = []
    for i in range(population_size // 4):
        parent1 = population_fitness[np.random.randint(len(population_fitness))][0]
        if np.random.random() > 0.5:
            parent2 = generate_full_tree(max_depth)
        else:
            parent2 = generate_grow_tree(max_depth)
        child = []
        if len(parent1) < len(parent2):
            for i in range(len(parent1)):
                if np.random.random() > 0.5:
                    child.append(parent1[i])
                else:
                    child.append(parent2[i])
            for j in range(len(parent1), len(parent2)):
                child.append(parent2[j])
        else:
            for i in range(len(parent2)):
                if np.random.random() > 0.5:
                    child.append(parent1[i])
                else:
                    child.append(parent2[i])
            for j in range(len(parent2), len(parent1)):
                child.append(parent1[j])
        new_population.append((child, fitness_function(x, y, child)))
    return new_population


generations = 100
def evolve_population(population_fitness, x, y):
    for i in range(generations):
        population_fitness = selection(population_fitness)
        new_population = crossover(population_fitness, x, y)
        population_fitness = population_fitness + new_population
        new_population = mutation(population_fitness, x, y)
        population_fitness = population_fitness + new_population
    return population_fitness

evolved_population = evolve_population(population_fitness, x, y)

### Best solution

In [ ]:
best_individual = sorted(evolved_population, key=lambda x: x[1])[0]

def break_tree(tree):
    for i in range(len(tree), -1, -1):
        if mean_squared_error(x, y, tree[:i]) == mean_squared_error(x, y, best_individual[0]):
            index = i
    return tree[:index]
        
best_individual_br = break_tree(best_individual[0])
print(f"Best individual: {best_individual_br}, Fitness: {best_individual[1]}, MSE: {mean_squared_error(x, y, best_individual[0])}")